In [1]:
from memory import MemoryBuffer
from policies import PPOTrainer
import tensorflow as tf
from tensorflow.layers import dense, conv2d, max_pooling2d, flatten
import numpy as np
from utils import preprocess_atari
import time
import gym

In [2]:
# Pacman network

model_name = 'breakout'
env = gym.make('Breakout-v0')
in_shape = [64, 64, 1]

obs = tf.placeholder(tf.float32, shape=[None]+in_shape)
conv1 = conv2d(obs, 16, 3, activation=tf.tanh)
pool2 = max_pooling2d(conv1, 2, 2)
conv2 = conv2d(pool2, 16, 3, activation=tf.tanh)
pool3 = max_pooling2d(conv2, 2, 2)
conv3 = conv2d(pool3, 16, 3, activation=tf.tanh)
flattened = flatten(conv3)
act_probs = dense(flattened, env.action_space.n)
softmax_probs = tf.nn.softmax(act_probs)

# v_conv1 = conv2d(obs, 16, 3, activation=tf.tanh)
# v_pool2 = max_pooling2d(v_conv1, 2, 2)
# v_conv2 = conv2d(v_pool2, 16, 3, activation=tf.tanh)
# v_pool3 = max_pooling2d(v_conv2, 2, 2)
# v_conv3 = conv2d(v_pool3, 16, 3, activation=tf.tanh)
# v_flattened = flatten(v_conv3)
value = dense(flattened, 1)

network = PPOTrainer(obs, softmax_probs, value, act_type='d', ppo_iters=80)

# Load network if previously saved
saver = tf.train.Saver()
try:
    saver.restore(network.sess, f'models/{model_name}.ckpt')
except:
    print('No model found, randomly initializing parameters')

/home/ejmejm/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


No model found, randomly initializing parameters


In [3]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    variable_parameters = 1
    for dim in shape:
        variable_parameters *= dim.value
    total_parameters += variable_parameters
print(total_parameters)

16325


In [7]:
n_episodes = 1000000
max_steps = 512
update_freq = 3
print_freq = 10

mb = MemoryBuffer()

In [ ]:
all_rewards = []

for episode in range(n_episodes):
    ep_reward = 0
    
    mb.start_rollout()
    obs = env.reset()
    for step in range(max_steps):
        obs = obs.squeeze()
        obs = preprocess_atari(obs)
        act = network.gen_act(obs)
        
        obs_next, rew, d, _ = env.step(act)
        ep_reward += rew
        
        if False:
            env.render()
            time.sleep(0.02)
        
        mb.record(obs, act, rew)
        obs = obs_next
        
        if d:
            break
            
    all_rewards.append(ep_reward)
            
    if episode % update_freq == 0 and episode != 0:
        network.train(mb.to_data())
        
        if episode % (update_freq * print_freq) == 0:
            print(f'Update #{episode // update_freq}, Reward: {np.mean(all_rewards[-update_freq*print_freq:])}')

Update #10, Reward: 3.3
Update #20, Reward: 3.0
Update #30, Reward: 4.9
Update #40, Reward: 3.9
Update #50, Reward: 4.266666666666667
Update #60, Reward: 4.766666666666667
Update #70, Reward: 4.866666666666666
Update #80, Reward: 5.033333333333333
Update #90, Reward: 5.8
Update #100, Reward: 5.2


In [ ]:
# Save the model
saver = tf.train.Saver()
saver.save(network.sess, 'models/breakout.ckpt')